# Bitcoin Key Pairs

In [1]:
# pip install ecdsa

## Manual Key Generation

In [2]:
import ecdsa
import binascii
import hashlib


# # Generate a private key
private_key_raw_object = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key raw:", (private_key_raw_object.to_string().hex()) + '\x01')

private_key_raw = private_key_raw_object.to_string()

# Get the private key
# private_key_hex = private_key_raw.to_string().hex()
# private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_raw, curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print("Private key:", private_key.to_string().hex())

# Get the public key
# public_key: ecdsa.VerifyingKey = private_key.get_verifying_key()
public_key = private_key.get_verifying_key()

# Print the public key in hexadecimal format
print(f"Public key compressed hex: {public_key.to_string("compressed").hex()}")
print(f"Public key uncompressed hex: {public_key.to_string("uncompressed").hex()}")


Private key raw: 9c6a04655be616bd82b0de98f1bfa7cc7c22b4f58fa2d5ecafde701dcbbe2eff
Private key: 9c6a04655be616bd82b0de98f1bfa7cc7c22b4f58fa2d5ecafde701dcbbe2eff
Public key compressed hex: 03fbcc9ee2e6fc0936c7ade15eb570bf202af0fab2851a8e3e8dbb1bd7727b07b0
Public key uncompressed hex: 04fbcc9ee2e6fc0936c7ade15eb570bf202af0fab2851a8e3e8dbb1bd7727b07b0c3e13f17e8ddbb5c02a1b3fb401e6e3c46bec80a96cfe706504eb91689a1760b


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

### Create a bitcoin address

In [3]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256: bytes = hashlib.sha256(public_key.to_string('compressed')).digest()
print("SHA256: ", sha256.hex())

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()
print("RIPEMD-160: ", hashed_public_key.hex())

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]
print("Address checksum: ", address_checksum.hex())

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum
print("Binary Bitcoin Address: ", binary_bitcoin_address.hex())

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address).decode()
print("Bitcoin address: ", bitcoin_address)

SHA256:  d6fe69a28c28d487055e7ee10d28f0ae8bc6715970d173d511c75d7145b069af
RIPEMD-160:  13a292090557a12933b9011111ae49927aefbabf
Address checksum:  47e5164b
Binary Bitcoin Address:  0013a292090557a12933b9011111ae49927aefbabf47e5164b
Bitcoin address:  12npbEnZ5VoBywYguSYXcTunkAXTVcaxQ2


### Verify a bitcoin address

In [4]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

Bitcoin address is valid: True


## Signing Messages

### Sign a message

In [5]:
def encode_varint(i: int) -> bytes:
    """
    Encode a potentially very large integer into varint bytes. The length should be
    specified in little-endian.

    https://bitcoin.org/en/developer-reference#compactsize-unsigned-integers
    """
    if i < 253:
        return bytes([i])
    elif i < 0x10000:
        return b"\xfd" + i.to_bytes(2, "little")
    elif i < 0x100000000:
        return b"\xfe" + i.to_bytes(4, "little")
    elif i < 0x10000000000000000:
        return b"\xff" + i.to_bytes(8, "little")
    else:
        raise ValueError("Integer is too large: %d" % i)

In [6]:
# Path: concepts_bitcoin_transactions.ipynb
import base64

private_key_raw_1 = private_key_raw_object
msg = b"Hello world!"
msg_size = encode_varint(len(msg))
message_hash = hashlib.sha256(hashlib.sha256(b'\x18Bitcoin Signed Message:\n' + msg_size + msg).digest()).digest()
print("Message hash:", message_hash.hex())

signature_ = private_key_raw_1.sign_deterministic(message_hash)
# Use the private key to sign a message
# message = bytes("Bitcoin Signed Message:\nHello world!", "utf-8")
# message = b"\x18Bitcoin Signed Message:\n" + msg_size + msg
# message_size = encode_varint(len(message))
# signature_ = private_key.sign_deterministic(hashlib.sha256(hashlib.sha256(message).digest()).digest())
# print("Signature:", signature_.hex())
print(f'Signature in b64: {base64.b64encode(bytes([28]) + signature_).decode('ascii')}')
print("Signature hex:", binascii.hexlify(signature_).decode('ascii'))

# Verify the signature
print(public_key.verify(signature_, message_hash))


Message hash: 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature in b64: HO1xOZ0Z+MlS4bl1l34pKRt+KMvflOjsvA1RRAP1AH3BzOcl0vWxEDPFnZ4Dtz5KghhnnNxihsAq+ftvpw0E4xk=
Signature hex: ed71399d19f8c952e1b975977e29291b7e28cbdf94e8ecbc0d514403f5007dc1cce725d2f5b11033c59d9e03b73e4a8218679cdc6286c02af9fb6fa70d04e319
True


In [7]:
# Get the private key
# private_key_hex = 'your_private_key_here'
# private_key_bytes = bytes.fromhex(private_key_hex)
# private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

# The message to sign
# message = 'your_message_here'

# Sign the message
signature = private_key_raw_1.sign(message_hash)
print(f"Signature: {signature.hex()}")

signature_b64 = base64.b64encode(bytes([28]) + signature).decode('ascii')
print(f"Signature b64: {signature_b64}")

# Print the signature in base64 format
# signature_b64 = base64.b64encode(signature).decode('ascii')
# signature_b64 = base64.b64decode(signature)
# print(f"Signature b64: {signature_b64}")

# Get the public key
public_key: ecdsa.VerifyingKey = private_key_raw_1.get_verifying_key()
print(f"Public key: {public_key.to_string('compressed').hex()}")


Signature: e62d0ef1f946ce48f1d2bb9981ff0c33e63982380cbd94f6599857864ecfb1f673b7592bb0792368ea24eea575a160fc25e7d70e777a83847c17b951c613078a
Signature b64: HOYtDvH5Rs5I8dK7mYH/DDPmOYI4DL2U9lmYV4ZOz7H2c7dZK7B5I2jqJO6ldaFg/CXn1w53eoOEfBe5UcYTB4o=
Public key: 03fbcc9ee2e6fc0936c7ade15eb570bf202af0fab2851a8e3e8dbb1bd7727b07b0


### Verify a message

In [8]:
# Use the public key to verify a message
# Verify the signature
try:
    public_key.verify(signature, message_hash)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Signature valid


# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [9]:
# pip install pybitcoin

In [10]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

Package appears damaged and does not work with python 3.12

In [11]:
# pip install bitcoin-utils

In [12]:
# from bitcoinutils.keys import PrivateKey, PublicKey

# # Generate a private key
# private_key = PrivateKey()
# print(f"Private key: {private_key.to_wif(compressed=True)}")

# # get the public key
# public_key: PublicKey = private_key.get_public_key()
# print(f"Public key: {public_key.to_hex(compressed=True)}")

# # get the bitcoin address
# bitcoin_address = public_key.get_address()
# print(f"Bitcoin address: {bitcoin_address.to_string()}")
# print(f'Hash160: {bitcoin_address.to_hash160()}')



Sign a message with a private key and verify the signature with the public key.

In [13]:
# message: str = "Hello"
# signature = private_key.sign_message(message)
# print(f"Signature: {signature}")
# # print(f"Signature valid? {public_key.verify(message, signature)}")
# print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

### python-bitcoinlib

In [14]:
# pip install python-bitcoinlib

In [15]:
import secrets
import base64
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress, CBitcoinAddress
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage
from bitcoin.core import x, lx, b2x, b2lx, Hash160

message_: str = "Hello world!"

# Generate a private key
private_key_ = CBitcoinSecret.from_secret_bytes(secrets.token_bytes(32))
print(f"Private key: {private_key_}")

# Generate a public key
public_key_ = private_key_.pub
print(f"Public key: {public_key_.hex()}")

# Generate a bitcoin address
bitcoin_address_ = P2PKHBitcoinAddress.from_pubkey(public_key_)
# bitcoin_address_ = CBitcoinAddress(bitcoin_address_)
print(f"Bitcoin address: {bitcoin_address_}")

# Sign message
bitcoin_msg = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_msg.magic} {bitcoin_msg.message} {bitcoin_msg.GetHash().hex()}")

signature_ = SignMessage(private_key_, bitcoin_msg)
print(f"Signature: {signature_.hex()}")
print(f'signature in b64: {base64.b64decode(signature_).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_, bitcoin_msg, signature_)}")


Private key: KykTXTqL8PKiEehYpGoZRicgXtzMEpCUbVVfaerDrDmVWvFHgCMY
Public key: 03a4895267173c744c59a0b20d752923626a83eebc7572b4c0d96d1bb5c87e6669
Bitcoin address: 1JVBF8LyqRUBQG4Nm2SMqqgwr87iGaEX5r
Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature: 48366451762f43335258756c4c594f532b4e495336475068487a424a2f6d6671526e44554d6c3670536f6b735252673954496a6c30654b6d45587434615a6e7056475146786b4a4e4c2f596f37326c38394552486171493d
signature in b64: 1fa750bff0b7457ba52d8392f8d212e863e11f3049fe67ea4670d4325ea94a892c45183d4c88e5d1e2a6117b786999e9546405c6424d2ff628ef697cf444476aa2
Signature valid? True


In [16]:
# verify the handmade message from the top
# print(f'Private Key BAse58 Encoded: {base58.b58encode(private_key_raw + b"\x01")}')
print(f'Private key not encoded: {private_key_raw.hex() + "01"}')
# bitccoin_private_key = CBitcoinSecret.from_secret_bytes(private_key_raw)
bitccoin_private_key = CBitcoinSecret.from_secret_bytes(bytes.fromhex(private_key_raw.hex() + "01"), 0)
bitccoin_private_key_bytes = bitccoin_private_key

# print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()[:-2]}")
print(f"Bitcoin private key: {bitccoin_private_key_bytes.hex()}")

# bitcoin_public_key_bytes = bitccoin_private_key_bytes.pub
# print(f'Public key base58 Encoded: {base58.b58encode(private_key.get_verifying_key().to_string('compressed'))}')
print(f'bitcoin public not encoded: {private_key.get_verifying_key().to_string('compressed').hex()}')

def compress_public_key(public_key: bytes) -> bytes:
    x = public_key[:32]
    y = public_key[32:]
    if int.from_bytes(y, 'big') % 2 == 0:
        return b'\x02' + x
    else:
        return b'\x03' + x
    
# bitcoin_public_key_bytes = compress_public_key(private_key_raw.get_verifying_key().to_string())
bitcoin_public_key_bytes = private_key.get_verifying_key().to_string('compressed')
print(f"Bitcoin public key: {bitcoin_public_key_bytes.hex()}")

print(f"Bitcoin address from top: {bitcoin_address}")
bitcoin_address_bytes = P2PKHBitcoinAddress.from_pubkey(bytes.fromhex(bitcoin_public_key_bytes.hex()))
print(f"Bitcoin address from pubkey: {bitcoin_address_bytes}")

# bitcoin_address_bytes = P2PKHBitcoinAddress.from_bytes(bytes.fromhex(bitcoin_public_key_bytes.hex()), 0)
# print(f"Bitcoin address: {bitcoin_address_bytes}")


Private key not encoded: 9c6a04655be616bd82b0de98f1bfa7cc7c22b4f58fa2d5ecafde701dcbbe2eff01
Bitcoin private key: 9c6a04655be616bd82b0de98f1bfa7cc7c22b4f58fa2d5ecafde701dcbbe2eff01
bitcoin public not encoded: 03fbcc9ee2e6fc0936c7ade15eb570bf202af0fab2851a8e3e8dbb1bd7727b07b0
Bitcoin public key: 03fbcc9ee2e6fc0936c7ade15eb570bf202af0fab2851a8e3e8dbb1bd7727b07b0
Bitcoin address from top: 12npbEnZ5VoBywYguSYXcTunkAXTVcaxQ2
Bitcoin address from pubkey: 12npbEnZ5VoBywYguSYXcTunkAXTVcaxQ2


In [17]:

bitcoin_message_bytes = BitcoinMessage(message_)
print(f"Bitcoin message: {bitcoin_message_bytes.magic} {bitcoin_message_bytes.message} {bitcoin_message_bytes.GetHash().hex()}")

print(f'signature from top: {signature_.decode()}')
# print(f'signature from top b64: {base64.b64encode(signature)}')
# signature_bytes = signature.__bytes__()
signature_bytes_b64 = base64.b64encode(bytes([32]) + signature_).hex()
print(f"Signature from top b64: {signature_bytes_b64}")

#sign message using bitcoin library
signature_bytes = SignMessage(bitccoin_private_key, bitcoin_message_bytes)
print(f"Signature bytes: {signature_bytes.decode()}")

# print(f'signature_bytes_hex: {base64.b64encode(signature_bytes).hex()}')
print(f'signature bytes decoded: {base64.b64decode(signature_bytes).hex()}')

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_bytes, bitcoin_message_bytes, signature_bytes)}")

Bitcoin message: b'Bitcoin Signed Message:\n' b'Hello world!' 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
signature from top: H6dQv/C3RXulLYOS+NIS6GPhHzBJ/mfqRnDUMl6pSoksRRg9TIjl0eKmEXt4aZnpVGQFxkJNL/Yo72l89ERHaqI=
Signature from top b64: 494567325a4646324c304d7a556c68316245785a54314d72546b6c544e6b645161456836516b6f7662575a78556d3545565531734e6e42546232747a556c4a6e4f56524a616d77775a557474525668304e474661626e4257523146476547744b546b7776575738334d6d77344f555653534746785354303d
Signature bytes: H85KUiAXT9g+jCMaaT9G5CA2icrAPOoEVwtpYRJIVY4TDp0mggweZI3VFkPx0k5mg6YANzHASxVg+i2hJFc7eKY=
signature bytes decoded: 1fce4a5220174fd83e8c231a693f46e4203689cac03cea04570b69611248558e130e9d26820c1e648dd51643f1d24e6683a6003731c04b1560fa2da124573b78a6
Signature valid? True


In [18]:
# Get the private key
private_key_hex = private_key_raw.hex()
private_key_bytes = binascii.unhexlify(private_key_hex)
private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)
print(f"Private key: {private_key.to_string().hex()}")

# Get the public key
public_key_hex = private_key.get_verifying_key().to_string('compressed').hex()
public_key_bytes = binascii.unhexlify(public_key_hex)
public_key = ecdsa.VerifyingKey.from_string(public_key_bytes, curve=ecdsa.SECP256k1)
print(f"Public key: {public_key.to_string('compressed').hex()}")

# Get the bitcoin address
bitcoin_address = P2PKHBitcoinAddress.from_pubkey(public_key_bytes)
print(f"Bitcoin address: {bitcoin_address}")

# The message to sign
message = bitcoin_message_bytes.GetHash()
print(f"Message: {message.hex()}")

# Sign the message
signature = private_key.sign(message)

# Print the signature in hexadecimal format
print(f"Signature: {binascii.hexlify(signature).decode()}")
print(f"Signature ascii: {binascii.hexlify(signature).decode('ascii')}")

signature_hex = binascii.hexlify(signature).decode('ascii')

# The signature
signature_unhex = binascii.unhexlify(signature_hex.encode('ascii'))
print(f"Signature unhex: {signature_unhex}")

# signature from b64
# signature_from_b64 = base64.b64decode(signature_hex)
# print(f"Signature from b64: {signature_from_b64}")

# signature base64
signature_b64 = base64.b64encode(bytes([32]) + signature_unhex).decode('ascii')
print(f"Signature b64: {signature_b64}")

# Verify the signature
try:
    public_key.verify(signature_unhex, message)
    print("Signature valid")
except ecdsa.BadSignatureError:
    print("Signature invalid")

Private key: 9c6a04655be616bd82b0de98f1bfa7cc7c22b4f58fa2d5ecafde701dcbbe2eff
Public key: 03fbcc9ee2e6fc0936c7ade15eb570bf202af0fab2851a8e3e8dbb1bd7727b07b0
Bitcoin address: 12npbEnZ5VoBywYguSYXcTunkAXTVcaxQ2
Message: 9b45fcc99ed84a7051317cc9c804b0235925ea1d9117cb9eb49b22efc3df28b3
Signature: 7a52932848c460969d37acee64101a489ae1b09b19cb371a272b382091650d0d550a81deeb36496d190646efb34fb512ab4d1c54075baf54aa2c363a55210bb2
Signature ascii: 7a52932848c460969d37acee64101a489ae1b09b19cb371a272b382091650d0d550a81deeb36496d190646efb34fb512ab4d1c54075baf54aa2c363a55210bb2
Signature unhex: b"zR\x93(H\xc4`\x96\x9d7\xac\xeed\x10\x1aH\x9a\xe1\xb0\x9b\x19\xcb7\x1a'+8 \x91e\r\rU\n\x81\xde\xeb6Im\x19\x06F\xef\xb3O\xb5\x12\xabM\x1cT\x07[\xafT\xaa,6:U!\x0b\xb2"
Signature b64: IHpSkyhIxGCWnTes7mQQGkia4bCbGcs3GicrOCCRZQ0NVQqB3us2SW0ZBkbvs0+1EqtNHFQHW69Uqiw2OlUhC7I=
Signature valid


In [19]:
# pip install pybitcoin

In [20]:
# !pip install bitcoin

# from pybitcoin import privtopub, ecdsa_raw_sign, encode_sig

# # Get the private key and message
# private_key = 'your_private_key_here'
# message = 'your_message_here'

# # Get the public key
# public_key = privtopub(private_key)

# # Sign the message
# r, s = ecdsa_raw_sign(message, private_key)

# # Calculate the recovery id
# recovery_id = 27 + (public_key[64] % 2)

# # Encode the signature in compact format
# compact_signature = encode_sig(recovery_id, r, s)

# print(f"Compact signature: {compact_signature}")

In [21]:
import hashlib
import ecdsa
import binascii
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage

def sign_message(private_key_hex, message):
    private_key_bytes = binascii.unhexlify(private_key_hex)
    private_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

    magic_message = b'\x18Bitcoin Signed Message:\n' + len(message).to_bytes(1, 'big') + message.encode()
    magic_message_hash = hashlib.sha256(hashlib.sha256(magic_message).digest()).digest()

    signature = private_key.sign(magic_message_hash)
    print(f"Signature: {signature}")
    # return binascii.hexlify(signature).decode()
    return signature

private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1, hashfunc=hashlib.sha256)
print(f"Private key: {private_key.to_string()}")

private_key_hex = private_key.to_string().hex()
print(f"Private key hex: {private_key_hex}")

message = 'Hello world!'
print(f"Message: {message}")

signature = sign_message(private_key_hex, message)
print(f"Signature hex: {signature.hex()}")

signature_b64 = base64.b64encode(bytes([32]) + signature).decode()
print(f"Signature b64: {signature_b64}")

# Verify the signature
def verify_signature(public_key_hex, signature, message):
    public_key_bytes = binascii.unhexlify(public_key_hex)
    public_key = ecdsa.VerifyingKey.from_string(public_key_bytes, curve=ecdsa.SECP256k1)

    magic_message = b'\x18Bitcoin Signed Message:\n' + len(message).to_bytes(1, 'big') + message.encode()
    magic_message_hash = hashlib.sha256(hashlib.sha256(magic_message).digest()).digest()

    return public_key.verify(signature, magic_message_hash)

public_key = private_key.get_verifying_key()
public_key_formatted = '04' + public_key.to_string().hex()
print(f"Public key formatted: {'04' + public_key.to_string().hex()}")

public_key_hex = public_key.to_string('compressed').hex()
print(f"Public key compressed hex: {public_key_hex}")

public_key_uncompressed = public_key.to_string('uncompressed').hex()
print(f"Public key uncompressed: {public_key_uncompressed}")

print(f"Signature valid (Check from pubkey)? {verify_signature(public_key_hex, signature, message)}")

# verify the signature using the python-bitcoinlib library
bitcoin_message = BitcoinMessage(message)
print(f"Bitcoin message: {bitcoin_message.magic} {bitcoin_message.message} {bitcoin_message.GetHash().hex()}")

bitcoin_private_key = CBitcoinSecret.from_secret_bytes(bytes.fromhex(private_key_hex), 0)
print(f"Bitcoin private key: {bitcoin_private_key}")
print(f"Bitcoin private key Hex: {bitcoin_private_key.hex()}")

# bitcoin_public_from_bitcoin_private = bitcoin_private_key.pub

bitcoin_public_key = P2PKHBitcoinAddress.from_pubkey(bytes.fromhex(public_key_hex))
print(f"Bitcoin public key: {bitcoin_public_key}")
print(f'bitcoin public key hex: {bitcoin_public_key.to_bytes().hex()}')
# print(f"Bitcoin public key Hex: {bitcoin_public_key.to_scriptPubKey().hex()}")

bitcoin_public_key = bitcoin_private_key.pub
print(f"Bitcoin public key: {bitcoin_public_key}")
print(f"Bitcoin public key hex: {bitcoin_public_key.hex()}")

bitcoin_address = P2PKHBitcoinAddress.from_pubkey(bitcoin_public_key)
print(f"Bitcoin address: {bitcoin_address}")

bitcoin_signed_message = SignMessage(bitcoin_private_key, bitcoin_message)
print(f"Bitcoin signed message: {bitcoin_signed_message}")
print(f"Bitcoin signed message base64decode: {base64.b64decode(bitcoin_signed_message).hex()}")

print(f"Signature valid? {VerifyMessage(bitcoin_address, bitcoin_message, bitcoin_signed_message)}")


Private key: b'8\xab\x85\x82\x1c\xceb\xfc\x97\xc3__\xa7\x8f\x82\x05\x1c\xe4f\xb0{o/S\xb1\xad\x08\xe2\xca\xe3\xb3e'
Private key hex: 38ab85821cce62fc97c35f5fa78f82051ce466b07b6f2f53b1ad08e2cae3b365
Message: Hello world!
Signature: b'\xe1#\xb3=\x83\xa99\x97\xae2\xedl\xfc\x10\xe5A\xf1\x1dN{F0\x9d3\xe2z\x03!-\xe4\x0f\xa5\xbd\xc0\xfd\x1fZ\xd1)\xb6\x97\xaa\x8fJ!z\xfeh\xfag"aD%q\xbeo\xb8\xbf\xe8-\x8e\xf5b'
Signature hex: e123b33d83a93997ae32ed6cfc10e541f11d4e7b46309d33e27a03212de40fa5bdc0fd1f5ad129b697aa8f4a217afe68fa672261442571be6fb8bfe82d8ef562
Signature b64: IOEjsz2DqTmXrjLtbPwQ5UHxHU57RjCdM+J6AyEt5A+lvcD9H1rRKbaXqo9KIXr+aPpnImFEJXG+b7i/6C2O9WI=
Public key formatted: 049292d2b6178267267aee6be56312ec546325cab4b19db2165a8b74860d717c456b610da352ba0cb6cd398051d44341acf8e4fcf1fdd9e1648ce3f76db2a696d3
Public key compressed hex: 039292d2b6178267267aee6be56312ec546325cab4b19db2165a8b74860d717c45
Public key uncompressed: 049292d2b6178267267aee6be56312ec546325cab4b19db2165a8b74860d717c456b610da352b